In [1]:
from dask.optimization_ast import ASTFunction, ASTDaskBuilder
import ast
import astor
import pickle

In [2]:
import dask.array as da
a = da.arange(4, chunks=2)
a = ((a + 1) * ((a > 2) + 2)).sum()
builder = ASTDaskBuilder(a.dask, a.__dask_keys__()[0])
func = ASTFunction(builder.tree, builder.constant_kwargs)
func()

24

In [3]:
print(func.source)

import dask.array.chunk
import dask.array.core
import numpy.core.fromnumeric


def _ast_compiled():
    dsk_arange = dask.array.chunk.arange(0, 2, 1, 2, 'int64')
    dsk_add = dsk_arange + 1
    dsk_gt = dsk_arange > 2
    dsk_add_1 = dsk_gt + 2
    dsk_mul = dsk_add * dsk_add_1
    dsk_sum = numpy.core.fromnumeric.sum(dsk_mul, axis=(0,), keepdims=True,
        dtype='int64')
    dsk_arange_1 = dask.array.chunk.arange(2, 4, 1, 2, 'int64')
    dsk_add_2 = dsk_arange_1 + 1
    dsk_gt_1 = dsk_arange_1 > 2
    dsk_add_3 = dsk_gt_1 + 2
    dsk_mul_1 = dsk_add_2 * dsk_add_3
    dsk_sum_1 = numpy.core.fromnumeric.sum(dsk_mul_1, axis=(0,), keepdims=
        True, dtype='int64')
    dsk_sum_2 = numpy.core.fromnumeric.sum(dask.array.core._concatenate2([
        dsk_sum, dsk_sum_1], axes=(0,)), dtype='int64', axis=(0,), keepdims
        =False)
    return dsk_sum_2



In [4]:
pickle.loads(pickle.dumps(func))()

24

In [5]:
import pprint
pprint.pprint(dict(a.dask))
#a.visualize()

{('add-33cba7fa3ec2aff46cefd3edb3d7a8d3', 0): (<built-in function add>,
                                               ('arange-a6743cea2a905a0cf2f23016494fe90a',
                                                0),
                                               1),
 ('add-33cba7fa3ec2aff46cefd3edb3d7a8d3', 1): (<built-in function add>,
                                               ('arange-a6743cea2a905a0cf2f23016494fe90a',
                                                1),
                                               1),
 ('add-57936cca728ab097b21206a93fea4d36', 0): (<built-in function add>,
                                               ('gt-d18a367d4137311a7a456e4468544ba7',
                                                0),
                                               2),
 ('add-57936cca728ab097b21206a93fea4d36', 1): (<built-in function add>,
                                               ('gt-d18a367d4137311a7a456e4468544ba7',
                                               

In [ ]:
c = a.dask['sum-aggregate-e2baf80ba9e0ac683a5dc8a3792f43c4',][0]

In [ ]:
c.first, c.funcs

In [ ]:
ast_source = astor.to_source(ast_code)
print(ast_source)

In [ ]:
%timeit pickle.loads(pickle.dumps(ast_code))

In [ ]:
ast_exec = compile(ast_code, filename='<ast>', mode='exec')
%timeit pickle.loads(pickle.dumps(ast_exec))

In [ ]:
%timeit pickle.loads(pickle.dumps(ast_source))

In [ ]:
%timeit compile(ast_source, filename='<ast>', mode='exec')

In [ ]:
%timeit pickle.loads(pickle.dumps(ast_source))

In [ ]:
astor.to_source(ast.Module(body=[ast.UnaryOp(ast.UAdd(), ast.Name('x', ast.Load()))]))